# "Historical" Tweets

In [25]:
import pandas as pd
import tweepy
import shutil
import jsonpickle
import json
import datetime

from pandas.io.json import json_normalize
# import flatten_json

## Part 1: Prepare API credentials
**Step 1: Authenticate account and tokens through tweepy**

In [26]:
import config

In [27]:
# authenticate account with tweepy
auth = tweepy.OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_key, config.access_secret)

# create API object to pull data from twitter - and pass in code!
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


# now we should be free to make twitter api calls!

**Step 2: Verify API is working with your account**

In [28]:
# check if program is working - output is your twitter name
user = api.me()
print (user.name)

amy taylor


**Step 3: Use geo_search API to get the `place_id` for a particular location**
<br>granularity = neighborhood (default) , city , admin or country
- EX: when query = 'Boston', the place_id is the same for neighborhood or city (empty for country)

In [29]:
#  Determine the city, country, or location you want
places = api.geo_search(query="Chicago", 
                        granularity="city"
                                          )
place_id = places[0].id
print('place_id is: ',place_id)

place_id is:  1d9a5370a355ab0c


***List of search queries made:***
<br>NOTE: there must be no space between place and place_ID

    
|searchQuery| place ID |granularity| file | # of tweets| extended tweet? |
|---|---|---|---|---|---|
|'place:96683cc9126741d1 road closed' | USA |country| ./PoGo_USA_Tutorial.json20190114_7_1_36.json | 142 | - |
|'Detroit road closed'|NA |NA | ./historical_20190114_7_36.json | 2 |- |
| 'Boston road closed' | NA |NA| ./historical_20190114_11_41.json | 1 |- |
| 'place:67b98f17fdcf20be road closed' | Boston | city/neigh | ./historical_20190114_11_55.json | 0|- |
| 'place:1c69a67ad480e1b1 road closed' | Houston | city/neigh | ./historical_20190114_11_59.json| 15|- |
| 'place:300bcc6e23a88361 road closed' | Seattle | city/neigh | ./historical_20190114_12_2.json | 3|- |
| 'Houston road closed' | NA | NA | ./historical_20190114_12_2.json | 9|- |
| 'Chicago road closed' | Chicago | city | ./historical_20190114_23_1.json | 1|- |
| 'place:1d9a5370a355ab0c road closed' | NA |NA | NA | 0|- |
|'place:96683cc9126741d1 road closed' | USA |city| ./historical_20190115_14_57.json | 210 | yes |


## Part 2: Download Archived Tweets with the Search API
**Step 1: Use Search API to download tweets based on our query search terms, and save tweets in a json file**
 - Include a place ID in the query if necessary
 - Other search params that are optional: since='2019-01-03',until='2019-01-11'
 
useful resource: http://www.dealingdata.net/2016/07/23/PoGo-Series-Tweepy/

In [30]:
searchQuery = 'place:96683cc9126741d1 road closed'

tweetCount = 0

#Open a text file to save the tweets to
with open('../data/AT_historical/historical.json', 'w') as f:

    #Tell the Cursor method: we want to use Search API (api.search), and our query
    for tweet in tweepy.Cursor(api.search,q=searchQuery,
                               tweet_mode='extended'    #comment out extended tweet if desired
                              ).items() :         

        #Verify the tweet has place info before writing 
        if tweet.place is not None:
            
            #Write the JSON format to the text file, and add one to the number of tweets we've collected
            f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
            tweetCount += 1

    #Display how many tweets we have collected
    print("Downloaded {0} tweets".format(tweetCount))

Downloaded 210 tweets


**Step 2: Append a timestamp to the end of the json file name**

In [31]:
def file_conversion():
    #create a timestamp
    now = datetime.datetime.now()
    month = '0'+str(now.month)
    day = str(now.day)
    hour = str(now.hour)
    minute = str(now.minute)
    now_str = str(now.year)+month+day+'_'+hour+'_'+minute
    
    
    #replace the name of our file with a new timestamped filename
    dest = '../data/AT_historical/historical_' + now_str + ".json"
    shutil.move('../data/AT_historical/historical.json', dest)
    
    with open(dest, "r") as f:
        status = f.readlines()
        jsons = []
        for ind in status:
            jsons.append(json.loads(ind))
    return jsons

In [32]:
file_conversion()

[{'contributors': None,
  'coordinates': {'coordinates': [-80.32491, 40.81398], 'type': 'Point'},
  'created_at': 'Tue Jan 15 22:55:22 +0000 2019',
  'display_text_range': [0, 123],
  'entities': {'hashtags': [{'indices': [40, 51], 'text': 'PaTurnpike'},
    {'indices': [91, 99], 'text': 'traffic'}],
   'symbols': [],
   'urls': [{'display_url': 'bit.ly/14Tzdd2',
     'expanded_url': 'http://bit.ly/14Tzdd2',
     'indices': [100, 123],
     'url': 'https://t.co/SL0Oqn0Vyr'}],
   'user_mentions': []},
  'favorite_count': 0,
  'favorited': False,
  'full_text': 'Road construction, right lane closed in #PaTurnpike on PATP EB between PA-18 and Cranberry #traffic https://t.co/SL0Oqn0Vyr',
  'geo': {'coordinates': [40.81398, -80.32491], 'type': 'Point'},
  'id': 1085309495755112448,
  'id_str': '1085309495755112448',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_

**Step 3: Add the newly created json file to the list below for easy access later**

In [10]:
# Previously created json files can be accessed from this list

# json_df = pd.read_json("../data/AT_historical/PoGo_USA_Tutorial.json20190114_7_1_36.json", lines = True)
json_df = pd.read_json("../data/AT_historical/historical_20190114_7_36.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190114_11_55.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190114_11_59.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190114_12_2.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190114_12_6.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190114_23_1.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190115_14_57.json", lines = True)


# Part 3: Explore Tweets from json file
### Option A: Tweets --> json file --> dataframe

In [21]:
json_df.columns

Index(['contributors', 'coordinates', 'created_at', 'entities',
       'extended_entities', 'favorite_count', 'favorited', 'geo', 'id',
       'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'metadata',
       'place', 'possibly_sensitive', 'quoted_status', 'quoted_status_id',
       'quoted_status_id_str', 'retweet_count', 'retweeted', 'source', 'text',
       'truncated', 'user'],
      dtype='object')

In [22]:
# print(json_df.shape)
# json_df.head(10)
# json_df.loc[:,"place":]

**View all texts from a tweet**

In [23]:
# view all entries in a column
list = json_df.loc[:, 'text']
# print(list)
for i in range(len(list)):
    print(i, list[i])
    print("--------")

0 Road construction, left lane closed in #Albuquerque on Tijeras Ave EB west of 3rd St #traffic https://t.co/roJyGryxFq
--------
1 Road construction. right lanes closed in #Pima on I-10 EB at Ruthrauff Rd #traffic https://t.co/bc4TAizExm
--------
2 Road construction, shoulder closed in #ElPaso on I 10 Both EB/WB from Executive Ctr Blvd to Sunland Park Dr #traffic https://t.co/nZ2qPHc0XJ
--------
3 Ughhh at the dentist for a cleaning and the sidewalks are closed. HOW DO YOU CLOSE A SIDEWALK. Like...am I supposed to walk in the road lol
--------
4 Road constructions. two right lanes closed in #Pima on I-10 EB at Ruthrauff Rd #traffic https://t.co/bc4TAizExm
--------
5 All eastbound lanes are closed due to snow and ice on the road. in #Valyermo on Angeles Crest Hwy EB between CA-39 and Big Pines Hwy
--------
6 State Rd 44 is closed at Sugar Creek Rd near the Johnson County/Shelby County Line. Shelby County Sheriff’s Officer… https://t.co/lXZj9ippFt
--------
7 Due to road conditions, schoo

Explore attributes of the json file

In [24]:
def PrintMembers(obj):
    for attribute in dir(obj):
        
        #We don't want to show built in methods of the class
        if not attribute.startswith('__'):
            print(attribute)
            
PrintMembers(tweet)

_api
_json
author
contributors
coordinates
created_at
destroy
entities
favorite
favorite_count
favorited
geo
id
id_str
in_reply_to_screen_name
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
is_quote_status
lang
metadata
parse
parse_list
place
possibly_sensitive
retweet
retweet_count
retweeted
retweets
source
source_url
text
truncated
user


In [25]:
# shrink the dataframe down to the only necessary columns
columns = ['coordinates', 'created_at', 'geo', 'place', 'text', 'user']
df = json_df.loc[:, columns]
df

,coordinates,created_at,geo,place,text,user
0,"{'coordinates': [-106.65, 35.08653], 'type': '...",2019-01-14 14:56:55,"{'coordinates': [35.08653, -106.65], 'type': '...","{'attributes': {}, 'bounding_box': {'coordinat...","Road construction, left lane closed in #Albuqu...","{'contributors_enabled': False, 'created_at': ..."
1,"{'coordinates': [-111.0295, 32.29418], 'type':...",2019-01-14 14:31:30,"{'coordinates': [32.29418, -111.0295], 'type':...","{'attributes': {}, 'bounding_box': {'coordinat...",Road construction. right lanes closed in #Pima...,"{'contributors_enabled': False, 'created_at': ..."
2,"{'coordinates': [-106.52, 31.79295], 'type': '...",2019-01-14 14:28:12,"{'coordinates': [31.79295, -106.52], 'type': '...","{'attributes': {}, 'bounding_box': {'coordinat...","Road construction, shoulder closed in #ElPaso ...","{'contributors_enabled': False, 'created_at': ..."
3,None,2019-01-14 14:08:05,None,"{'attributes': {}, 'bounding_box': {'coordinat...",Ughhh at the dentist for a cleaning and the si...,"{'contributors_enabled': False, 'created_at': ..."
4,"{'coordinates': [-111.0295, 32.29418], 'type':...",2019-01-14 13:49:35,"{'coordinates': [32.29418, -111.0295], 'type':...","{'attributes': {}, 'bounding_box': {'coordinat...",Road constructions. two right lanes closed in ...,"{'contributors_enabled': False, 'created_at': ..."
5,"{'coordinates': [-117.8511, 34.35686], 'type':...",2019-01-14 13:13:00,"{'coordinates': [34.35686, -117.8511], 'type':...","{'attributes': {}, 'bounding_box': {'coordinat...",All eastbound lanes are closed due to snow and...,"{'contributors_enabled': False, 'created_at': ..."
6,None,2019-01-14 13:06:22,None,"{'attributes': {}, 'bounding_box': {'coordinat...",State Rd 44 is closed at Sugar Creek Rd near t...,"{'contributors_enabled': False, 'created_at': ..."
7,None,2019-01-14 12:51:37,None,"{'attributes': {}, 'bounding_box': {'coordinat...","Due to road conditions, schools and offices, i...","{'contributors_enabled': False, 'created_at': ..."
8,None,2019-01-14 12:29:51,None,"{'attributes': {}, 'bounding_box': {'coordinat...",For those going to @BenLippenSchool down Monti...,"{'contributors_enabled': False, 'created_at': ..."
9,"{'coordinates': [-97.32055, 32.713], 'type': '...",2019-01-14 11:44:34,"{'coordinates': [32.713, -97.32055], 'type': '...","{'attributes': {}, 'bounding_box': {'coordinat...",Road construction. two left lanes closed in #F...,"{'contributors_enabled': False, 'created_at': ..."


### Explore some features of the df

In [26]:
# how do we extract just the coordinates?
# for k, v in df['geo']:
#     print(k)                   
# --------> coordinates
#     print(v) 
#  --------> type

# for k in df['geo']:
#     print(k)
# ---------> {'coordinates': [29.68057, -95.38122], 'type': 'Point'}

# for k, v in df['geo'].items():
#     print(k)  
# #     ----------> 0, 1, ...
#     print(v)
# #     ---------> {'coordinates': [29.68057, -95.38122], 'type': 'Point'}

# for k, v in df['geo'].k():
#     print(k)

# for k, v in df['place'].items():
#     for v1 in v:
#         print(v1)       

In [27]:
df['geo'][0]

{'coordinates': [35.08653, -106.65], 'type': 'Point'}

In [29]:
# see all the nested information within place
list = json_df.loc[:, 'place']
# print(list)
for i in range(len(list)):
    print(i, list[i])
    print("--------")

0 {'attributes': {}, 'bounding_box': {'coordinates': [[[-106.7916912, 35.0158912], [-106.473745, 35.0158912], [-106.473745, 35.218114], [-106.7916912, 35.218114]]], 'type': 'Polygon'}, 'contained_within': [], 'country': 'United States', 'country_code': 'US', 'full_name': 'Albuquerque, NM', 'id': '813a485b26b8dae2', 'name': 'Albuquerque', 'place_type': 'city', 'url': 'https://api.twitter.com/1.1/geo/id/813a485b26b8dae2.json'}
--------
1 {'attributes': {}, 'bounding_box': {'coordinates': [[[-111.083219, 32.057802], [-110.747928, 32.057802], [-110.747928, 32.320979], [-111.083219, 32.320979]]], 'type': 'Polygon'}, 'contained_within': [], 'country': 'United States', 'country_code': 'US', 'full_name': 'Tucson, AZ', 'id': '013379ee5729a5e6', 'name': 'Tucson', 'place_type': 'city', 'url': 'https://api.twitter.com/1.1/geo/id/013379ee5729a5e6.json'}
--------
2 {'attributes': {}, 'bounding_box': {'coordinates': [[[-106.634874, 31.6206683], [-106.199987, 31.6206683], [-106.199987, 31.966068], [-1

## Option B. Tweets --> json --> parsed --> nested dataframe

Convert json file into a ...

In [15]:
# Previously created json files can be accessed from this list

json_df = pd.read_json("../data/AT_historical/PoGo_USA_Tutorial.json20190114_7_1_36.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190114_7_36.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190114_11_55.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190114_11_59.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190114_12_2.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190114_12_6.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190114_23_1.json", lines = True)

In [17]:
tweets_data = []
notParsed = []
tweets_file = open("../data/AT_historical/PoGo_USA_Tutorial.json20190114_7_1_36.json","r")
for line in tweets_file:    
    if line.strip():    
        try:
            tweet=json.loads(line)
            tweets_data.append(tweet)
        except:
            notParsed.append(line)
            continue
print(len(tweets_data))
print('Could not parse: ', len(notParsed))

143
Could not parse:  0


Unpack the nested columns

In [21]:
tweet_cols = ['coordinates', 'created_at', 
#                'full_text',
              'geo', 'id', 'place', 'user']

sample_tweets_dict = [{col:tweet[col] for col in tweet_cols } for tweet in tweets_data]

# To flatten all nested dictionaries
# DOES NOT FLATTEN LISTS
# Look for any instances of nested lists during cleaning
sample_tweets_df = pd.io.json.json_normalize(sample_tweets_dict)

In [19]:
sample_tweets_df

,coordinates,coordinates.coordinates,coordinates.type,created_at,geo,geo.coordinates,geo.type,id,place.bounding_box.coordinates,place.bounding_box.type,...,user.profile_text_color,user.profile_use_background_image,user.protected,user.screen_name,user.statuses_count,user.time_zone,user.translator_type,user.url,user.utc_offset,user.verified
0,NaN,"[-106.65, 35.08653]",Point,Mon Jan 14 14:56:55 +0000 2019,NaN,"[35.08653, -106.65]",Point,1084826703724249089,"[[[-106.7916912, 35.0158912], [-106.473745, 35...",Polygon,...,333333,True,False,TotalTrafficABQ,22351,None,none,https://t.co/0ksDF2WEvB,None,False
1,NaN,"[-111.0295, 32.29418]",Point,Mon Jan 14 14:31:30 +0000 2019,NaN,"[32.29418, -111.0295]",Point,1084820307217731584,"[[[-111.083219, 32.057802], [-110.747928, 32.0...",Polygon,...,333333,True,False,TotalTrafficTUC,44176,None,none,None,None,False
2,NaN,"[-106.52, 31.79295]",Point,Mon Jan 14 14:28:12 +0000 2019,NaN,"[31.79295, -106.52]",Point,1084819476888125440,"[[[-106.634874, 31.6206683], [-106.199987, 31....",Polygon,...,333333,True,False,TotalTrafficELP,64341,None,none,None,None,False
3,NaN,NaN,NaN,Mon Jan 14 14:08:05 +0000 2019,NaN,NaN,NaN,1084814410621898753,"[[[-86.044756, 41.426657], [-85.9629436, 41.42...",Polygon,...,000000,True,False,ShelbyGray_,84724,None,none,https://t.co/6V64xuiS9G,None,False
4,NaN,"[-111.0295, 32.29418]",Point,Mon Jan 14 13:49:35 +0000 2019,NaN,"[32.29418, -111.0295]",Point,1084809756940910592,"[[[-111.083219, 32.057802], [-110.747928, 32.0...",Polygon,...,333333,True,False,TotalTrafficTUC,44176,None,none,None,None,False
5,NaN,"[-117.8511, 34.35686]",Point,Mon Jan 14 13:13:00 +0000 2019,NaN,"[34.35686, -117.8511]",Point,1084800548413554688,"[[[-124.482003, 32.528832], [-114.131212, 32.5...",Polygon,...,0C3E53,True,False,TotalTrafficLA,239386,None,none,https://t.co/JwhaVm9kQy,None,False
6,NaN,NaN,NaN,Mon Jan 14 13:06:22 +0000 2019,NaN,NaN,NaN,1084798880204632065,"[[[-86.348441, 39.631677], [-85.937404, 39.631...",Polygon,...,000000,True,False,AlannaMartella,16639,None,none,https://t.co/iicj4cTiXj,None,True
7,NaN,NaN,NaN,Mon Jan 14 12:51:37 +0000 2019,NaN,NaN,NaN,1084795167230574593,"[[[-75.689007, 38.275414], [-75.6022644, 38.27...",Polygon,...,333333,True,False,EFitzSolava,764,None,none,https://t.co/7aivx6ANqZ,None,False
8,NaN,NaN,NaN,Mon Jan 14 12:29:51 +0000 2019,NaN,NaN,NaN,1084789691709308928,"[[[-83.353955, 32.04683], [-78.499301, 32.0468...",Polygon,...,333333,True,False,JohnCombs_BL,602,None,none,https://t.co/mouOOMAqQq,None,False
9,NaN,"[-97.32055, 32.713]",Point,Mon Jan 14 11:44:34 +0000 2019,NaN,"[32.713, -97.32055]",Point,1084778296087375872,"[[[-97.538285, 32.569477], [-97.033542, 32.569...",Polygon,...,42301C,True,False,TotalTrafficDFW,275709,None,none,http://t.co/vThEPeuicN,None,False


In [23]:
# sample_tweets_df['user.screen_name'].value_counts()